In [1]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


#  Домашнее задание по NLP # 2 [100 баллов] 
## Анализ тональности твитов

Данные для задания – [коллекция русскоязычных твитов](http://study.mokoron.com/), среди которых есть заведомо отрицательные и заведомо положительные. Ваша задача - научиться как можно лучше отделять заведомо положительные от заведомо отрицательных.

Загрузите 2 csv файла: [с положительными твитами](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0) и [с отрицательными](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0)

## Часть 1. Обучение модели word2vec [40 баллов]

0. Объедините положительные и отрицательные твиты в один массив
1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. 
2. Обучите по коллекции предложений word2vec
3. Приведите несколько удачных и неудачных примеров решения стандартных тестов для word2vec:
    * тест на определение ближайших слов
    * тест на аналогии (мужчина – король : женщина – королева)
    * тест на определение лишнего слова.
    
4. Постройте визуализацию TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов 

Ссылка на примеры визуализаций: https://towardsdatascience.com/game-of-thrones-word-embeddings-does-r-l-j-part-2-30290b1c0b4b

База данных состоит из 12 столбцов:

- id: уникальный номер сообщения в системе twitter;
- tdate: дата публикации сообщения (твита);
- tmane: имя пользователя, опубликовавшего сообщение;
- ttext:  текст сообщения (твита);
- ttype: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный, отрицательный, нейтральный);
- trep: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;
- trtw: число ретвитов
- tfav: число сколько раз данное сообщение было добавлено в избранное другими пользователями;
- tstcount: число всех сообщений пользователя в сети twitter;
- tfol: количество фоловеров пользователя (тех людей, которые читают пользователя);
- tfrien: количество друзей пользователя (те люди, которых читает пользователь);
- listcount: количество листов-подписок в которые добавлен твиттер-пользователь.

In [4]:
columns = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'tfav', 'trtw', 'tstcount', 'tfol', 'tfrien', 'listcount']

In [5]:
negative = pd.read_csv('negative.csv', sep=';', encoding='utf-8', error_bad_lines=False, names=columns)

In [6]:
positive = pd.read_csv('positive.csv', sep=';', encoding='utf-8', error_bad_lines=False, names=columns)

In [7]:
negative.head()

,id,tdate,tmane,ttext,ttype,trep,tfav,trtw,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [8]:
positive.head()

,id,tdate,tmane,ttext,ttype,trep,tfav,trtw,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


### 0. Объедините положительные и отрицательные твиты в один массив

In [9]:
df = pd.concat([positive, negative])

In [10]:
df = df.reset_index().drop(columns=['index'])

In [11]:
import sklearn.utils
df = sklearn.utils.shuffle(df)

### 1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. 

In [12]:
import re
regex = re.compile("[А-Яа-я]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return ""


df['ttext'] = df.ttext.apply(lambda x: x.lower())
df['words_only'] = df.ttext.apply(words_only)

In [13]:
from nltk.corpus import stopwords
sw = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д', 'сч', 'хотя']

In [14]:
def remove_stopwords(text, mystopwords = sw):
    try:
        return [token for token in text if not token in mystopwords]
    except:
        return ""

In [15]:
df['words_only_without_sw']= df['words_only'].apply(remove_stopwords) 

In [16]:
from pymorphy2 import MorphAnalyzer

pm2 = MorphAnalyzer()


def lemmatize(text, mystem=pm2):
    try:
        return [pm2.parse(word)[0].normal_form for word in text]
    except:
        return " "

In [17]:
df['lemmas'] = df['words_only_without_sw'].apply(lemmatize) 

In [18]:
df.to_csv('df_twit_lemmas.csv', encoding='utf-8', index=False)

In [19]:
df[['lemmas']].head()

,lemmas
143892,"[уснуть, автобус, провтыкать, остановка, зашиб..."
219386,"[число, метр, платон, право]"
90214,"[жёлтый, луна, уровень, крыша, посмотреть, окно]"
222569,"[обувь, новый, купить, пятка, кровь, натереть]"
164526,"[конец, четверть, беда, зарабатывать, оценка, ..."


In [20]:
from gensim.models import Word2Vec

C:\Users\Arsenii\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 2. Обучите по коллекции предложений word2vec

In [21]:
model = Word2Vec(df.lemmas, size=100, window=3, min_count=20, workers=4)

In [22]:
model.save("word2v_twit.model")

C:\Users\Arsenii\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [23]:
model = Word2Vec.load("word2v_twit.model")

C:\Users\Arsenii\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### 3. Приведите несколько удачных и неудачных примеров решения стандартных тестов для word2vec.

In [24]:
model.most_similar("школа")

[('шк', 0.8591850399971008),
 ('колледж', 0.8097853660583496),
 ('универ', 0.739445686340332),
 ('учёба', 0.7242656946182251),
 ('треня', 0.7223893404006958),
 ('репетитор', 0.7220944762229919),
 ('музыкалка', 0.7167025208473206),
 ('репетиция', 0.7081418037414551),
 ('никуда', 0.6994667649269104),
 ('тренировка', 0.6868067979812622)]

In [25]:
model.most_similar("ум")

[('далёкий', 0.5923976898193359),
 ('поп', 0.5874745845794678),
 ('щ', 0.5867449045181274),
 ('приложить', 0.5820604562759399),
 ('посуда', 0.5745445489883423),
 ('мяч', 0.5702961683273315),
 ('пацан', 0.5694011449813843),
 ('ру', 0.5693413019180298),
 ('бабка', 0.5587953329086304),
 ('ппц', 0.5562995076179504)]

In [26]:
model.most_similar(positive=["соцсеть","время"], negative=["твиттер"])

[('надежда', 0.7624943256378174),
 ('дежурство', 0.6928868889808655),
 ('мучение', 0.6735677719116211),
 ('казалось', 0.6696842312812805),
 ('надолго', 0.6660622954368591),
 ('энергия', 0.66511070728302),
 ('минутка', 0.6634714603424072),
 ('смыть', 0.660553514957428),
 ('финансовый', 0.6575322151184082),
 ('расслабиться', 0.6562843322753906)]

**Неудачный пример:**

In [27]:
model.most_similar(positive=["король","женщина"], negative=["мужчина"])

[('снизу', 0.9204174280166626),
 ('квадрат', 0.8945657014846802),
 ('слюни', 0.892627477645874),
 ('тереть', 0.8920769691467285),
 ('упор', 0.8901880979537964),
 ('справа', 0.8871051073074341),
 ('мария', 0.8869766592979431),
 ('копипаста', 0.8868688941001892),
 ('стрелка', 0.8851778507232666),
 ('увеличить', 0.8833971619606018)]

In [28]:
model.doesnt_match("город церковь кремль река работа".split())

'работа'

In [29]:
model.doesnt_match("твиттер радость гулять жить смеяться".split())

'твиттер'

### 4. Постройте визуализацию TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов

In [30]:
from nltk import FreqDist

In [31]:
top_words = []

fd = FreqDist()
for text in tqdm(df.lemmas):
    fd.update(text)
for i in fd.most_common(1000):
    top_words.append(i[0])

100%|██████████████████████████████████████████████████████████████████████| 226834/226834 [00:02<00:00, 108390.79it/s]


In [32]:
top_words_vec_1 = model[top_words[:500]]
top_words_vec_2 = model[top_words[:100]]

In [33]:
from sklearn.manifold import TSNE

In [34]:
tsne = TSNE(n_components=2, random_state=0)

top_words_tsne_1 = tsne.fit_transform(top_words_vec_1)
top_words_tsne_2 = tsne.fit_transform(top_words_vec_2)

In [35]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne_1[:,0],
                                    x2=top_words_tsne_1[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

In [36]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne_2[:,0],
                                    x2=top_words_tsne_2[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

- **Можно выделить, например, кластер связанный со словами о сне. "утро, ночь, спать, просыпаться, сон" и тд.**
- **С уроками "урок, история, английский, русский".**
- **С погодой "улица, холодный, погода, снег, зима"**
- **С соцсетями "вк, твиттер, твит, лента".**
- **И т.д.**

## Часть 2. Анализ тональности [60 баллов]

Отделите 25% положительных и 25% отрицательных твитов как тестовую выборку. Обучите классификатор, отличающий положительно окрашенные твиты от отрицательно окрашенных на обучающей выборке (оставшиеся 75%) и оцените их accuracy на выборке тестовой. А лучше несколько классификаторов, устроенных по-разному. Если у вас получилось несколько очень похожих классификаторов (скажем, один и тот же класс из sklearn с разными гиперпараметрами), они будут считаться не разными методами, а разными версиями одного метода и учитываться будет только лучшая версия. Тем не менее, по возможности не удаляйте их из ноутбука! Очень интересно, как вы пришли к результату, к которому вы пришли.

Можно использовать предобученные модели. К обучающей выборке можно добавить какие-нибудь ещё данные, которые вы сами где-то найдёте, а можно наоборот - вообще её не использовать (сделать классификатор, основанный на вручную написанных правилах).

Если у вас будет хотя бы один работающий классификатор, ваша оценка за часть 2 будет вычислена по формуле:

$$
\min( 10 + \sum_i (a_i - 50); 60 )
$$

где $a_i$ - accuracy одного из ваших (разных!) методов на тестовой выборке.

В этом задании (при желании) можно применить все знания, полученные за курс:
- Морфология и синтаксический парсинг
- Тематические модели
- Bag of words и TF/IDF
- Дистрибутивная семантика
- Классические классификаторы (RandomForest)
- Нейронные классификаторы
- Языковые модели
- Attention
- Извлечение терминов

А так же задавать в слаке любые вопросы

Как видно из формулы, у вас есть выбор - брать количеством или качеством :)

Удачи и да пребудет с вами BERT.

In [37]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [38]:
X = df.drop(columns=['ttype', 'tdate', 'tmane', 'ttext', 'words_only', 'words_only_without_sw'])
y = df['ttype']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=22, shuffle=True)

In [40]:
def sent_to_vec(sent, w2v = model):
    vec = np.array([w2v[w] for w in sent if w in w2v])
    if len(vec):
        return(np.mean(vec, axis=0))
    else:
        return(np.zeros(100))

In [41]:
%time X_train['mean_vec'] = X_train.lemmas.apply(lambda x: sent_to_vec(x))

Wall time: 16.6 s


In [42]:
%time X_test['mean_vec'] = X_test.lemmas.apply(lambda x: sent_to_vec(x))

Wall time: 5.7 s


In [43]:
X_train = X_train.drop(columns=['lemmas'])

In [44]:
X_test = X_test.drop(columns=['lemmas'])

In [45]:
from sklearn.linear_model import LogisticRegression

In [46]:
from xgboost import XGBClassifier

In [47]:
clf = LogisticRegression()

In [48]:
X_train = np.array(X_train.mean_vec.tolist())
X_test= np.array(X_test.mean_vec.tolist())

In [49]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [50]:
y_pred = clf.predict(X_test)

In [51]:
from sklearn.metrics import accuracy_score

In [52]:
accuracy_score(y_test, y_pred)

0.6634572995468091

In [53]:
clf = XGBClassifier()

In [54]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [55]:
y_pred = clf.predict(X_test)

In [56]:
accuracy_score(y_test, y_pred)

0.6580613306529828

In [57]:
def join(text):
    return ' '.join(text)

In [58]:
df['joined_lemmas'] = df.lemmas.apply(join)

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['joined_lemmas'][:10000]).toarray()

In [60]:
from sklearn.feature_extraction.text import TfidfTransformer

X = TfidfTransformer().fit_transform(X)
X.shape

(10000, 15056)

In [61]:
feature_names = vectorizer.get_feature_names()

In [62]:
X = pd.DataFrame(X.toarray())

In [63]:
y = df['ttype'][:10000]

In [64]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22)

In [65]:
clf = XGBClassifier(
                    max_depth=5,
                    n_jobs=-1,
                    n_estimators=101,
                    min_samples_split=200,
                    learning_rate=0.079,
                    max_features='log2',
                    subsample=0.6578947368421052
)

In [66]:
clf.fit(X=X_train, y=y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       learning_rate=0.079, max_delta_step=0, max_depth=5,
       max_features='log2', min_child_weight=1, min_samples_split=200,
       missing=None, n_estimators=101, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=0.6578947368421052, verbosity=1)

In [67]:
y_test_predict = clf.predict(X_test)

In [68]:
accuracy_score(y_test, y_test_predict)

0.6168

In [69]:
stopw = set(
    [w for w in stopwords.words(['russian', 'english'])]
    + [u'это', u'году', u'года', u'также', u'етот',
       u'которые', u'который', u'которая', u'поэтому',
       u'весь', u'свой', u'мочь', u'eтот', u'например',
       u'какой-то', u'кто-то', u'самый', u'очень', u'несколько',
       u'источник', u'стать', u'время', u'пока', u'однако',
       u'около', u'немного', u'кроме', u'гораздо', u'каждый',
       u'первый', u'вполне', u'из-за', u'из-под',
       u'второй', u'нужно', u'нужный', u'просто', u'большой',
       u'хотеть', u'начать', u'должный', u'новый', u'день',
       u'метр', u'получить', u'далее', u'именно', u'апрель',
       u'сообщать', u'разный', u'говорить', u'делать',
       u'появиться', u'2016',
       u'2015', u'получить', u'иметь', u'составить', u'дать', u'читать',
       u'ничто', u'достаточно', u'использовать',
       u'принять', u'практически',
       u'находиться', u'месяц', u'достаточно', u'что-то', u'часто',
       u'хотеть', u'начаться', u'делать', u'событие', u'составлять',
       u'остаться', u'заявить', u'сделать', u'дело',
       u'примерно', u'попасть', u'хотя', u'лишь', u'первое',
       u'больший', u'решить', u'число', u'идти', u'давать', u'вопрос',
       u'сегодня', u'часть', u'высокий', u'главный', u'случай', u'место',
       u'конец', u'работать', u'работа', u'слово', u'важный', u'сказать',
       u'то', u'по', u'на', u'как', u'что', u'это', u'или', u'не', u'нет', u'за'])
    

text_dict = {}
i = 0
for comment in df['joined_lemmas'][:15000]:
    text_list = list(set(comment.split()))
    text_list_cleaned = []
    for word in text_list:
        if word not in list(stopw):
            text_list_cleaned.append(word)     
    text_dict[i] = text_list_cleaned
    i += 1

In [70]:
N_topics = 10

In [71]:
%%time
from gensim.corpora import TextCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

class ListTextCorpus(TextCorpus):

    def get_texts(self):
        for doc in self.input:
            yield doc
                
mycorp = ListTextCorpus(input=text_dict.values())
justlda = LdaModel(
    corpus=mycorp, num_topics=N_topics, passes=90)

Wall time: 4min 7s


In [72]:
print('LdaModel performance')
topic_description = {}
for i in range(N_topics):
    terms = justlda.get_topic_terms(i)
    topic_description[i] = ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms))
    print(i, ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms)))

LdaModel performance
0 сидеть спасибо пора год помнить прям прийтись хороший давно понимать
1 любить почему рождение человек бывать правда хороший днём спасибо говорят
2 видеть получиться телефон зима б ходить туда р год жизнь
3 болеть плохо ахах взять деньга голова з готовый вс москва
4 вообще твой скучать смотреть найти понять друг папа любимый никто
5 утро добрый вечер вс круто хороший думать блин проснуться снег
6 настроение надеяться приехать забыть завтра год дом бог половина жаль
7 год нормальный человек новое вообще пятница устать ребёнок подарок учиться
8 писать ждать знать каникулы узнать наверное написать твит смотреть играть
9 спать пойти ночь школа завтра ахи сон норма кома нг


In [73]:
dummy_dict = {key: 0 for key in range(N_topics)}

comment_topics_dict_2 = {
    id: dict(list(dummy_dict.items()) + justlda.get_document_topics(mycorp.dictionary.doc2bow(normalized_words)))
    for id, normalized_words in text_dict.items()
}
check_pd_2 = pd.DataFrame.from_dict(comment_topics_dict_2, orient='index')
check_pd_2.head(15)

,0,1,2,3,4,5,6,7,8,9
0,0.016670,0.183328,0.016667,0.016667,0.016667,0.683334,0.016667,0.016667,0.016667,0.016668
1,0.033333,0.033376,0.033333,0.033333,0.033333,0.033333,0.033333,0.699958,0.033333,0.033333
2,0.014286,0.014287,0.178224,0.014287,0.014286,0.440544,0.281228,0.014286,0.014287,0.014286
3,0.016667,0.016668,0.016667,0.634482,0.016669,0.016671,0.016668,0.232174,0.016668,0.016667
4,0.183335,0.016667,0.016667,0.183337,0.016667,0.016669,0.016667,0.016667,0.016667,0.516660
5,0.157138,0.014287,0.014286,0.409046,0.014286,0.014286,0.014286,0.333815,0.014286,0.014286
6,0.014287,0.014286,0.014286,0.014286,0.014286,0.014286,0.724726,0.014286,0.014286,0.160987
7,0.016670,0.683332,0.016667,0.016667,0.016668,0.016668,0.016667,0.016667,0.183329,0.016667
8,0.137500,0.012500,0.012500,0.504459,0.012503,0.012500,0.145536,0.012502,0.012501,0.137500
9,0.016667,0.016667,0.016668,0.016668,0.300100,0.183333,0.016667,0.016667,0.016667,0.399898


In [74]:
check_pd_2.shape

(15000, 10)

In [75]:
check_pd_2.columns = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9', 'class_10']

In [76]:
X = df[:15000].drop(columns=['ttype', 'tdate', 'tmane', 'ttext', 'words_only', 'words_only_without_sw', 'lemmas', 'joined_lemmas'])
y = df[:15000]['ttype']

In [77]:
X['class_1'] = check_pd_2['class_1']
X['class_2'] = check_pd_2['class_2']
X['class_3'] = check_pd_2['class_3']
X['class_4'] = check_pd_2['class_4']
X['class_5'] = check_pd_2['class_5']
X['class_6'] = check_pd_2['class_6']
X['class_7'] = check_pd_2['class_7']
X['class_8'] = check_pd_2['class_8']
X['class_9'] = check_pd_2['class_9']
X['class_10'] = check_pd_2['class_10']

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22)

In [79]:
clf = XGBClassifier()

In [80]:
clf.fit(X=X_train, y=y_train)
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)

In [81]:
accuracy_score(y_test, y_test_predict)

0.9285333333333333

In [82]:
columns = ['sent_to_vec + logreg', 'sent_to_vec + xgboost', 'TF/IDF + xgboost', 'LDA (n=10) + xgboost']
accuracy = ['0.66', '0.66', '0.62', '0.93']

In [97]:
res = pd.DataFrame(accuracy, columns=['accuracy'], index=columns).T

## Таким образом, наши результаты:

In [98]:
res

,sent_to_vec + logreg,sent_to_vec + xgboost,TF/IDF + xgboost,LDA (n=10) + xgboost
accuracy,0.66,0.66,0.62,0.93
